In [1]:
import pandas as pd
import config
from pathlib import Path
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain.agents import create_sql_agent, initialize_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = config.GOOGLE_CREDENTIALS

In [2]:
# Creating Agent
llm = OpenAI(temperature=0, verbose=True, openai_api_key=config.OPENAI_API_KEY)

/Users/alejandromunoz/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [3]:
from langchain.sql_database import SQLDatabase
from sqlalchemy import create_engine

# Definir el proyecto y dataset
PROJECT_ID = "acoustic-env-452102-r3"
DATASET_ID = "olist"

# Crear la URI de conexión
BIGQUERY_URI = f"bigquery://{PROJECT_ID}/{DATASET_ID}"

# Crear el motor SQLAlchemy para BigQuery
engine = create_engine(BIGQUERY_URI)


In [4]:
# Conectar la base de datos a LangChain
db = SQLDatabase(engine)

/Users/alejandromunoz/Library/Python/3.9/lib/python/site-packages/google/cloud/bigquery/client.py:587: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


In [6]:
# Probar la conexión ejecutando una consulta en la tabla "olist.ecommerce_data"
query = "SELECT * FROM olist.olist_order_items LIMIT 5"
result = db.run(query)

In [7]:
agent_executor_sql = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [9]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Crear el prompt para mejorar la respuesta en español
template = """
Eres un asistente virtual especializado en eCommerce, diseñado para ayudar a los clientes de manera clara, amigable y útil.

Tu tarea es transformar la siguiente información técnica obtenida de una consulta SQL en una respuesta atractiva y comprensible para el usuario final. Asegúrate de:

1️⃣ **Hacer la respuesta fácil de entender** (evita términos técnicos innecesarios).
2️⃣ **Ser conversacional y cercano** (habla como un asistente amigable, no como un robot).
3️⃣ **Organizar la información** en una estructura clara (listas, párrafos cortos, etc.).
4️⃣ **Ofrecer contexto adicional** si es necesario (ejemplo: si el usuario pregunta por un producto, menciona beneficios clave).
5️⃣ **Finalizar con una llamada a la acción** si aplica (ejemplo: "¿Te gustaría que te ayude a encontrar más productos similares? 😊").

### 🔹 **Ejemplo de Transformación**
❌ **Entrada SQL sin formato:**
"El producto más vendido es el 'Smartphone X'. Se han vendido 10,000 unidades en los últimos 30 días."

✅ **Salida esperada (más clara y amigable):**
📢 **¡El Smartphone X es un éxito de ventas!** 🚀  
En los últimos 30 días, **10,000 personas** lo han comprado, lo que lo convierte en uno de los productos más populares de nuestra tienda.  
👉 **¿Quieres conocer más sobre sus características o encontrar accesorios compatibles?** 😊

---

### 🔹 **Aquí está la respuesta SQL que debes reformatear:**
{respuesta_sql}

🔄 **Devuelve la versión mejorada aquí:**
"""



In [10]:
prompt = PromptTemplate(template=template, input_variables=["respuesta_sql"])

# 2️⃣ Crear la cadena de traducción y reformateo
agent_executor_translator = LLMChain(llm=llm, prompt=prompt, verbose=True)

/Users/alejandromunoz/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [11]:
# 3️⃣ Función para ejecutar los agentes en secuencia
def preguntar_sql_en_espanol(pregunta):
    print(f"\n🔹 Pregunta original: {pregunta}")

    # 1. Ejecutar consulta SQL con el primer agente
    respuesta_sql = agent_executor_sql.run(pregunta)
    print(f"\n✅ Respuesta SQL: {respuesta_sql}")

    # 2. Traducir y reformatear con el segundo agente
    respuesta_final = agent_executor_translator.run(respuesta_sql)

    return respuesta_final

In [12]:
# 6️⃣ Prueba del flujo de agentes
pregunta_usuario = "¿Cual es el producto más caro vendido en la tienda?"
respuesta = preguntar_sql_en_espanol(pregunta_usuario)
print(respuesta)


🔹 Pregunta original: ¿Cual es el producto más caro vendido en la tienda?


> Entering new SQL Agent Executor chain...


/Users/alejandromunoz/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


Action: sql_db_list_tables
Action Input: olist_customers, olist_geolocation, olist_order_items, olist_order_payments I should query the olist_order_items table to get the product prices.
Action: sql_db_schema
Action Input: olist_order_items
CREATE TABLE `olist_order_items` (
	`order_id` STRING, 
	`order_item_id` INT64, 
	`product_id` STRING, 
	`seller_id` STRING, 
	`shipping_limit_date` STRING, 
	`price` FLOAT64, 
	`freight_value` FLOAT64
)

/*
3 rows from olist_order_items table:
order_id	order_item_id	product_id	seller_id	shipping_limit_date	price	freight_value
3ee6513ae7ea23bdfab5b9ab60bffcb5	1	8a3254bee785a526d548a81a9bc3c9be	96804ea39d96eb908e7c3afdb671bb9e	2018-05-04 03:55:26	0.85	18.23
6e864b3f0ec71031117ad4cf46b7f2a1	1	8a3254bee785a526d548a81a9bc3c9be	96804ea39d96eb908e7c3afdb671bb9e	2018-05-02 20:30:34	0.85	18.23
c5bdd8ef3c0ec420232e668302179113	2	8a3254bee785a526d548a81a9bc3c9be	96804ea39d96eb908e7c3afdb671bb9e	2018-05-07 02:55:22	0.85	22.3
*/ I should query the olist_order_i

In [13]:
respuesta

'\n📢 **¡Buenas noticias!** 🎉  \nEl producto que estás buscando tiene un precio de **$6735.0**.  \n👉 **¿Te gustaría ver más detalles sobre este producto o buscar opciones similares?** 😊'